# December 06, 2024

https://adventofcode.com/2024/day/06

In [3]:
from collections import defaultdict

In [10]:
def parse_input( lines ):
    rules = list()
    updates = list()
    for line in lines:
        if len(line) == 0:
            continue
        if "|" in line:
            x,y = line.split("|")
            rules.append( [int(x) for x in line.split("|")] )
        else:
            updates.append( [int(x) for x in line.split(",")] )

    return rules, updates

In [15]:
test = f'''....#.....
.........#
..........
..#.......
.......#..
..........
.#..^.....
........#.
#.........
......#...'''

test = test.split("\n")

In [18]:
fn = "../data/2024/06.txt"
with open(fn, "r") as file:
    text = file.readlines()
puzz = [line.strip() for line in text]

# Part 1

In [149]:
def step( guard, facing, obstacles, dim ):
    def check_bounds( pos, dim ):
        # return True if pos is within the dimensions
        if pos[0] < 0 or pos[0] >= dim[0] or pos[1] < 0 or pos[1] >= dim[1]:
            return False
        return True
    
    def check_space( pos, obstacles ):
        # return True if pos is free of obstacles
        return not pos[1] in obstacles[pos[0]]
    
    
    # before: after
    turns = {"^":">", ">":"v", "v":"<", "<":"^"}

    # find the target space
    if facing == "^":
        new_pos = [ guard[0]-1, guard[1] ]
    elif facing == ">":
        new_pos = [ guard[0], guard[1]+1 ]
    elif facing == "v":
        new_pos = [ guard[0]+1, guard[1] ]
    else:
        new_pos = [ guard[0], guard[1]-1 ]

    if not check_bounds(new_pos, dim):
        #print(f'''exit grid at {new_pos[0]}, {new_pos[1]}.''')
        return None, facing
    
    if check_space(new_pos, obstacles):
        guard = new_pos
    else:
        facing = turns[facing]
        #print(f'''hit obstacle at {new_pos[0]}, {new_pos[1]}. Now facing {facing}''')

    return guard, facing


def follow_path( guard, facing, obstacles, dim ):
    def track_step( pos, track ):
        if pos is not None:
            track[pos[0]].add( pos[1] )
    
    track = defaultdict(set)
    track_step( guard, track )

    while guard is not None:
        guard, facing = step( guard, facing, obstacles, dim )
        track_step(guard, track)

    steps = 0
    for val in track.values():
        steps += len(val)

    return steps


def part1( puzz ):

    dim = ( len(puzz), len(puzz[0]) )
    # key = row, value = set of columns with #
    obstacles = dict()

    # first parse the input
    for i, line in enumerate(puzz):
        obstacles[i] = set()
        for j, ch in enumerate(line):
            if ch == "#":
                obstacles[i].add(j)
            elif ch in ["^", ">", "v", "<"]:
                guard = [i,j]
                facing = ch
    # end parsing

    return follow_path( guard, facing, obstacles, dim )







In [49]:
test

['....#.....',
 '.........#',
 '..........',
 '..#.......',
 '.......#..',
 '..........',
 '.#..^.....',
 '........#.',
 '#.........',
 '......#...']

In [50]:
part1(test)

41

In [51]:
part1(puzz)

4559

# Part 2

wrong answers:
1658 -- too high

In [197]:
def follow_path2( guard, facing, obstacles, dim ):
    # if return_path is True, return the list of steps the guard takes
    # otherwise, return True if you're caught in a loop, otherwise return False
    def track_step( pos, facing, track ):
        if pos is not None:
            track[facing][pos[0]].add( pos[1] )
    
    # for part two, we need to know both previous position and bearing
    track = {"^":defaultdict(set),
             ">":defaultdict(set),
             "v":defaultdict(set),
             "<":defaultdict(set)
    }

    solutions = defaultdict(set)

    
    while guard is not None:
        old_guard=[*guard]    
        # caught in a loop!
        if guard[1] in track[facing][guard[0]]:
            #print("loop")
            return True, track

            
        track_step(guard, facing, track)
        guard, facing = step( guard, facing, obstacles, dim )
  


    #print("exit at ", *old_guard)
    return False, track


def get_next_space( pos, facing ):
    if facing == "^":
        return pos[0]-1, pos[1]
    elif facing == ">":
        return pos[0], pos[1]+1
    elif facing == "v":
        return pos[0]+1, pos[1]
    elif facing == "<":
        return pos[0], pos[1]-1
    
    raise( "illegal face!")

def part2( puzz ):
    
    dim = ( len(puzz), len(puzz[0]) )
    # key = row, value = set of columns with #
    obstacles = defaultdict(set)

    # first parse the input
    for i, line in enumerate(puzz):
        for j, ch in enumerate(line):
            if ch == "#":
                obstacles[i].add(j)
            elif ch in ["^", ">", "v", "<"]:
                guard = [i,j]
                facing = ch
    # end parsing

    loop, path = follow_path2( guard, facing, obstacles, dim )

    # try placing an obstacle at every step
    solutions = defaultdict(set)
    for key, rowdict in path.items():
        for row, colset in rowdict.items():
            for col in colset:
                new_obs = get_next_space( [row, col], key )
                # There's already an obstacle. Nothing to check!
                if new_obs[1] in obstacles[new_obs[0]]:
                    continue

                obstacles[new_obs[0]].add( new_obs[1] )

                # check for a loop
                looped, new_path = follow_path2( guard, facing, obstacles, dim )
                if looped:
                    solutions[new_obs[0]].add(new_obs[1])

                obstacles[new_obs[0]].remove( new_obs[1] )

    sol_count = 0
    for sol_set in solutions.values():
        sol_count += len(sol_set)
    return path, sol_count




In [203]:
path, sc = part2(test)
sc

6

In [207]:
path, sc = part2(puzz)
sc

1604

In [206]:
tot = 0
for f, rd in path.items():
    for r, cs in rd.items():
        tot += len(cs)

# how many options did I check?
# what % of all possible answers is that?
tot, tot/(130*130)

(5113, 0.30254437869822487)